In [8]:
import glob
import json
import os.path

jsonfile = 'data/fusion360/train_test.json'

bin_files = glob.glob('data/fusion360/dgl/*.bin')

names = [os.path.basename(f).split('.')[0] for f in bin_files]





In [9]:
from tqdm import tqdm

new_jsonfile = 'data/fusion360/train_test_new.json'
new_train_names = []
new_test_names = []
with open(jsonfile, 'r') as f:
    data = json.load(f)
    for name in tqdm(data['train']):
        if name in names:
            new_train_names.append(name)
    for name in tqdm(data['test']):
        if name in names:
            new_test_names.append(name)

100%|██████████| 1725/1725 [00:00<00:00, 29213.32it/s]


In [10]:
new_data = {'train': new_train_names, 'test': new_test_names}
with open(new_jsonfile, 'w') as f:
    json.dump(new_data, f)

In [4]:
from source.cadlib.macro import EXTENT_TYPE, EXTRUDE_OPERATIONS


def parse_extent_type(distance1, distance2):
    if distance2 == 128:
        return EXTENT_TYPE.index('OneSideFeatureExtentType')
    if distance1 == distance2:
        return EXTENT_TYPE.index('SymmetricFeatureExtentType')
    else:
        return EXTENT_TYPE.index('TwoSidesFeatureExtentType')


def parse_command_type(command_type):
    if command_type == 6:
        command_type = 4
    elif command_type == 7:
        command_type = 5
    elif command_type == 5:
        command_type = 3
    elif command_type == 4:
        raise ValueError('command type error', command_type)

    if command_type > 5:
        raise ValueError('command type error', command_type)
    return command_type


def parse_extrude_operation(extrude_operation):
    if extrude_operation == 0:
        return EXTRUDE_OPERATIONS.index('NewBodyFeatureOperation')
    elif extrude_operation == 1:
        return EXTRUDE_OPERATIONS.index('CutFeatureOperation')
    elif extrude_operation == 2:
        return EXTRUDE_OPERATIONS.index('IntersectFeatureOperation')
    elif extrude_operation < 0:
        return -1
    else:
        raise ValueError('extrude operation error', extrude_operation)


def single_cad_vec_convert(single_cad_vec):
    command_type = parse_command_type(single_cad_vec[0])
    new_params = single_cad_vec[1:15]

    u = parse_extent_type(single_cad_vec[13], single_cad_vec[14])
    if command_type != 5:
        u = -1
    b = parse_extrude_operation(single_cad_vec[19])
    new_params = np.concatenate([new_params, np.array([b, u])])

    assert len(new_params) == 16
    return np.concatenate([np.array([command_type]), new_params])


def cad_vec_convert(cad_vec):
    length = cad_vec.shape[0]
    new_cad_vecs = []
    for i in range(length):
        new_single_cad_vec = single_cad_vec_convert(cad_vec[i])
        new_cad_vecs.append(new_single_cad_vec)
    return np.array(new_cad_vecs)

100%|██████████| 20/20 [00:00<00:00, 1021.28it/s]


NameError: name 'init_display' is not defined